In [1]:
using JuliaDB, IterableTables, Dates, Printf, JLD2, FileIO, Plots, OnlineStats;

Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /Users/osx/buildbot/slave/package_osx64/build/src/./locks.h:138 [inlined]
jl_typeinf_end at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:2610
typeinf_ext at ./compiler/typeinfer.jl:577
typeinf_ext at ./compiler/typeinfer.jl:613
jfptr_typeinf_ext_1 at /Applications/Julia-1.1.app/Contents/Resources/julia/lib/julia/sys.dylib (unknown line)
_include_from_serialized at ./loading.jl:633
Core.CodeInfo(code=Array{Any, (2,)}[
  Expr(:using, Expr(:., :JuliaDB), Expr(:., :IterableTables), Expr(:., :Dates), Expr(:., :Printf), Expr(:., :JLD2), Expr(:., :FileIO), Expr(:., :Plots), Expr(:., :OnlineStats)),
  Expr(:return, nothing)], codelocs=Array{Int32, (2,)}[1, 1], method_for_inference_limit_heuristics=nothing, ssavaluetypes=2, linetable=Array{Any, (1,)}[Core.LineInfoNode(mod=Main, method=Symbol("top-level scope"), file=Symbol("In[1]"), line=1, inlined_at=0)], ssaflags=Array{UInt8, (0,)}[], sl

In [2]:
function makeCompDict(wcafile)
    compids = String[]
    compdates = Date[]
    lines = readlines(wcafile)
    for i = 2:length(lines) # skip header line
        line = lines[i]
        data = split(line, "\t")
        push!(compids, data[1])
        compdate = string(data[6], "-", data[7], "-", data[8])
        push!(compdates, Date(compdate, "y-m-d"))
    end
        compDict = Dict(zip(compids, compdates))
        sortedCompsArray = sort(collect(zip(values(compDict),keys(compDict))))
    
    return compDict, sortedCompsArray
end

Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /Users/osx/buildbot/slave/package_osx64/build/src/./locks.h:138 [inlined]
jl_typeinf_end at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:2610
typeinf_ext at ./compiler/typeinfer.jl:577
typeinf_ext at ./compiler/typeinfer.jl:613
jfptr_typeinf_ext_1 at /Applications/Julia-1.1.app/Contents/Resources/julia/lib/julia/sys.dylib (unknown line)
#invokelatest#1 at ./essentials.jl:742 [inlined]


makeCompDict (generic function with 1 method)

In [3]:
function genGlickoArray(comp, event, round)
    
    myresults = table(results[comp, event, round, :])
    mytable = table(select(myresults, (:personId, :pos)), pkey=:pos)
    
    output = []
    for i in 1:length(mytable), j in (i+1):length(mytable)
        o = [mytable[i].personId, mytable[j].personId, 1]
        o2 = [mytable[j].personId, mytable[i].personId, 0]
        push!(output, o)
        push!(output, o2)
    end
    
    return output
end

genGlickoArray (generic function with 1 method)

In [4]:
function genAllGlickoArrays(event)
    compDict, sortedCompsArray = makeCompDict("WCA_export_Competitions.tsv")
    allGlickoArrays = []
    lengthOfComps = length(sortedCompsArray)
    print("\r\r\r")
    for i in 1:length(sortedCompsArray)
        index = sortedCompsArray[i][1]
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "0"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "1"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "2"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "3"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "b"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "c"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "d"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "e"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "f"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "g"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "h"))
        print("\r $i / $lengthOfComps Comps  ||  $index")
    end
    output = []
    for i in 1:length(allGlickoArrays)
        if length(allGlickoArrays[i]) != 0
            push!(output, allGlickoArrays[i])
        end
    end
    return output
end

genAllGlickoArrays (generic function with 1 method)

In [5]:
function calcRating(p1,p2,sc, localratingdict, localRDdict, ratingdict, RDdict, voldict)
    tau = 0.5
    g(phi) = (1 + (3*(phi^2)*2)/(2*(pi^2)))^(-1/2)
    E(mu1, mu2, phi2) = 1/(1 + exp(-g(phi2)*(mu1 - mu2)))
    f(x) = (exp(x)*(delta^2 - phi1^2 - v - exp(x)))/(2*(phi1^2 + v + exp(x))^2) - (x-a)/(tau^2)
    
    r1 = ratingdict[p1]
    r2 = localratingdict[p2]
    RD1 = RDdict[p1]
    RD2 = localRDdict[p2]
    sigma1 = voldict[p1]
    
    mu1 = (r1 - 1500)/173.7178
    phi1 = RD1/173.7178
    mu2 = (r2 - 1500)/173.7178
    phi2 = RD2/173.7178
    
    v = ((g(phi2)^2)*E(mu1, mu2, phi2)*(1 - E(mu1, mu2, phi2)))^-1
    delta = v*(g(phi2)*(sc - E(mu1, mu2, phi2)))
    epsilon = 0.000001
    
    A = a = log(sigma1^2)
    if delta^2 > phi1^2 + v
        B = log(delta^2 - phi1^2 - v)
    else
        k = 1
        while f(a - k*tau) < 0
            k += 1
        end
        B = a - k*tau 
    end
    fA = f(A)
    fB = f(B)
    while abs(B-A) > epsilon
        C = A + ((A - B)*fA)/(fB - fA)
        fC = f(C)
        if fC*fB < 0 
            A = B
            fA = fB
        else
            fA = fA/2
        end
        B = C
        fB = fC
    end
    
    sigma1prime = exp(A/2)
    phistar = sqrt(phi1^2 + sigma1prime^2)
    phi1prime = 1/sqrt(1/(phistar^2) + 1/v)
    mu1prime = mu1 + (phi1prime^2)*(g(phi2)*(sc - E(mu1, mu2, phi2)))
   
    r1prime  = 173.7178*mu1prime + 1500
    RD1prime = 173.7178*phi1prime
    
    return r1prime, RD1prime, sigma1prime
end

calcRating (generic function with 1 method)

In [6]:
function generateRatings(data, RD, vol)
    playerids = []
    for i in 1:length(wca_persons)
        push!(playerids, wca_persons[i][1])
    end

    #reset ratings
    ratings = []
    RDs = []
    vols = []
    for i in 1:length(playerids)
        push!(ratings, 1500.0)
        push!(RDs, RD)
        push!(vols, vol)
    end

    ratingdict = Dict(zip(playerids, ratings))
    RDdict = Dict(zip(playerids, RDs))
    voldict = Dict(zip(playerids, vols));
    
    history_of_top100 = []
    
    print("\r\r\r")
    for i in 1:length(data)
        localratingdict = ratingdict
        localRDdict = RDdict
        ratingdict, RDdict, voldict = calcRatings(data[i], localratingdict, localRDdict, ratingdict, RDdict, voldict)
        #push!(history_of_top100, sort(collect(ratingdict), by=x->x[2], rev=true)[1:100])
        print("\r$i / ", length(data))
    end
    
    RatingList = sort(collect(zip(values(ratingdict),keys(ratingdict))), rev = true)
    
    return RatingList#, history_of_top100, ratingdict
end    

generateRatings (generic function with 1 method)

In [7]:
function calcRatings(data, localratingdict, localRDdict, ratingdict, RDdict, voldict)
    for i in 1:length(data)
        r1prime, RD1prime, sigma1prime = calcRating(data[i][1], data[i][2], data[i][3], localratingdict, localRDdict, ratingdict, RDdict, voldict)
        ratingdict[data[i][1]] = r1prime
        RDdict[data[i][1]] = RD1prime
        voldict[data[i][1]] = sigma1prime
    end
    return ratingdict, RDdict, voldict
end
    

calcRatings (generic function with 1 method)

In [8]:
function printRatings(RatingList)
    htb = @sprintf "%4s %-30s \t %-7s %-15s" "Rank" "Name" "Rating" "Citizen of"
    println()
    println(htb)
    for i in 1:length(RatingList)
        id = RatingList[i][2]
        name = persons[id, 1].name
        if length(name) > 30
            name = name[1:30]*"..."
        end
        rating = RatingList[i][1]
        country = persons[id, 1].countryId
        tb = @sprintf "%4d %30s \t %6.2f %-15s" i rpad(name, 30) rating rpad(country, 15)
        println(tb)
    end
end

printRatings (generic function with 1 method)

In [9]:
results = loadndsparse("WCA_export_Results.tsv"; delim = '\t',
          indexcols = [:competitionId, :eventId, :roundTypeId, :personId],
          colparsers=Dict("personId"=>String,"eventId"=>String,"competitionId"=>String, "roundTypeId"=>String, "pos" =>Int))

4-d NDSparse with 2020611 values (13 field named tuples):
    Dimensions#  colname        type
────────────────────────
1  competitionId  String
2  eventId        String
3  roundTypeId    String
4  personId       String
    Values#   colname                type
─────────────────────────────────
5   pos                    Int64
6   best                   Int64
7   average                Int64
8   personName             String
9   personCountryId        String
10  formatId               String
11  value1                 Int64
12  value2                 Int64
13  value3                 Int64
14  value4                 Int64
15  value5                 Int64
16  regionalSingleRecord   String
17  regionalAverageRecord  String

In [10]:
path_persons = "WCA_export_Persons.tsv"
wca_persons = loadtable(path_persons; delim = '\t', quotechar = '\\',
    colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

persons = loadndsparse(path_persons; delim = '\t', quotechar = '\\', indexcols = [:id, :subid],
    colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

In [11]:
function genTable(ratings)
    rankA = Int[]
    ratingA = Float64[]
    countryA = String[]
    nameA = String[]
    wcaidA = String[]
    for i in 1:length(ratings)
        push!(rankA, i)
        push!(ratingA, ratings[i][1])
        push!(countryA, persons[ratings[i][2], 1].countryId)
        push!(nameA, persons[ratings[i][2], 1].name)
        push!(wcaidA, ratings[i][2])
    end
    outTable = table(rankA, ratingA, countryA, nameA, wcaidA, 
                     names=[:Rank, :Rating, :Country, :Name, :WcaId])
    return outTable
end

genTable (generic function with 1 method)

In [12]:
function genCSV(event, date)
    glickoArraysThing = genAllGlickoArrays(event)
    volValue = 0.006*(15000/length(glickoArraysThing))
    ratings = generateRatings(glickoArraysThing, 35, volValue)
    filename = ""*event*"_"*date*".csv"
    FileIO.save(filename, genTable(ratings))
end

genCSV (generic function with 1 method)

In [13]:
function genAllCSV(date)
    genCSV("222", date)
    genCSV("333", date)
    genCSV("444", date)
    genCSV("555", date)
    genCSV("666", date)
    genCSV("777", date)
    genCSV("minx", date)
    genCSV("pyram", date)
    genCSV("skewb", date)
    genCSV("sq1", date)
    genCSV("clock", date)
    genCSV("333oh", date)
    genCSV("333bf", date)
    genCSV("333ft", date)
    genCSV("333fm", date)
    genCSV("444bf", date)
    genCSV("555bf", date)
end

genAllCSV (generic function with 1 method)

In [15]:
genAllCSV("Feb_2019")

 5474 / 5751 Comps  ||  2019-01-19103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331033310333103331

Excessive output truncated after 524296 bytes.

In [ ]:
thing333 = genAllGlickoArrays("333")

 5131 / 5751 Comps  ||  2018-10-12

In [ ]:
RatingList333 = generateRatings(thing333, 35, 0.006)

In [ ]:
@save "February_10_2019.jld2" thing333, RatingList333

In [64]:
#@save "December_27_2018.jld2" RatingList history_of_top100 ratingdict
#@load "December_27_2018.jld2" RatingList history_of_top100 ratingdict

In [81]:
printRatings(RatingList) #Pyraminx Cube (35, 0.012)


Rank Name                           	 Rating  Citizen of     
   1 Drew Brads                     	 3014.83 USA            
   2 Dominik Górny                  	 2908.46 Poland         
   3 Jules Desjardin                	 2906.32 France         
   4 Daniel Goodman                 	 2906.07 USA            
   5 Rafał Waryszak                 	 2862.82 Poland         
   6 Tymon Kolasiński               	 2834.16 Poland         
   7 Marko Striieshyn               	 2819.00 Ukraine        
   8 Livia Kleiner                  	 2803.22 USA            
   9 Aniket Das                     	 2801.58 India          
  10 Martin Vædele Egdal            	 2771.91 Denmark        
  11 Henry Savich                   	 2746.25 USA            
  12 Elvin Thorsen                  	 2731.38 Sweden         
  13 Marcus Kamen                   	 2726.79 USA            
  14 Robin Verstraten               	 2713.73 Netherlands    
  15 Samuel Simko                   	 2699.78 Slovakia       
  16 Ma

 203 Tomáš Nguyen                   	 2231.51 Vietnam        
 204 Jeffrey Draper                 	 2231.02 Canada         
 205 Chia-Liang Tai (戴嘉良)           	 2229.56 Taiwan         
 206 Nicholas Patterson             	 2229.48 Canada         
 207 Maciej Jabłoński               	 2228.43 Poland         
 208 Chris Mills                    	 2228.27 United Kingdom 
 209 Joseph Lee                     	 2227.52 USA            
 210 Cormac Farrell                 	 2226.89 United Kingdom 
 211 Andy Denney                    	 2225.06 USA            
 212 Łukasz Burliga                 	 2223.91 Poland         
 213 Jason Tang                     	 2223.90 Canada         
 214 Paul Taylor                    	 2222.99 USA            
 215 Joshua Feran                   	 2222.65 USA            
 216 Dawid Jasiński                 	 2221.26 Poland         
 217 Kaiyang Gao (高凯阳)              	 2220.04 China          
 218 Jēkabs Janovs                  	 2219.93 Latvia         
 219 Ram

StringIndexError: StringIndexError("Alejandro García-Carrasco Gómez", 30)

In [77]:
printRatings(RatingList) #2x2 Cube (35, 0.006)


Rank Name                           	 Rating  Citizen of     
   1 Antonie Paterakis              	 2821.16 Greece         
   2 Martin Vædele Egdal            	 2711.64 Denmark        
   3 Lucas Etter                    	 2685.27 USA            
   4 Jiekang Pan (潘杰康)              	 2669.51 China          
   5 Maciej Czapiewski              	 2653.54 Poland         
   6 Luke Van Laningham             	 2641.04 USA            
   7 Kevin Gerhardt                 	 2608.42 Germany        
   8 Daniel Rose-Levine             	 2591.34 USA            
   9 Rami Sbahi                     	 2589.90 USA            
  10 Jayden McNeill                 	 2584.62 Australia      
  11 Brian Johnson                  	 2584.52 USA            
  12 Yinghao Wang (王鹰豪)             	 2574.04 China          
  13 Will Callan                    	 2539.64 USA            
  14 Nicolás Sánchez                	 2530.17 USA            
  15 Eva Kato                       	 2526.79 USA            
  16 Ca

 162 Ivan Li Ka Leong (李嘉亮)         	 2182.31 Hong Kong      
 163 Vladimir Korovin               	 2182.04 Russia         
 164 Yan-Hung Lai (賴彥宏)             	 2181.75 Taiwan         
 165 Haichuan Du (杜海川)              	 2180.76 China          
 166 Kai-Wen Wang (王楷文)             	 2180.68 Taiwan         
 167 Louis Cormier                  	 2180.55 Canada         
 168 Martin Šebesta                 	 2180.28 Czech Republic 
 169 Gustavo Penaforte Brito        	 2179.38 Brazil         
 170 Nikhil Soares                  	 2179.15 India          
 171 Artem Yashkov                  	 2179.10 Russia         
 172 Joel Cetra                     	 2179.04 Germany        
 173 Thomas Schmidt                 	 2177.58 Denmark        
 174 Pavel Galaktionov              	 2177.52 Russia         
 175 Matěj Grohmann                 	 2175.41 Czech Republic 
 176 Sean Hartman                   	 2175.09 USA            
 177 Yu Tang (唐语)                   	 2174.74 China          
 178 Ant

StringIndexError: StringIndexError("Vladyslav Zhyhailo (Владислав Жигайло)", 30)

In [95]:
printRatings(RatingList) # 7x7 Cube (35,0.043)


Rank Name                           	 Rating  Citizen of     
   1 Max Park                       	 3055.36 USA            
   2 Feliks Zemdegs                 	 3001.92 Australia      
   3 Kevin Hays                     	 2919.16 USA            
   4 Lucas Wesche                   	 2910.67 Germany        
   5 Simon Stannek                  	 2894.63 Germany        
   6 Michał Halczuk                 	 2849.31 Poland         
   7 Abdelhak Kaddour               	 2758.27 France         
   8 Marco Vorländer                	 2735.12 Germany        
   9 Ciarán Beahan                  	 2730.30 Ireland        
  10 Anyu Zhang (张安宇)               	 2725.77 China          
  11 Wong Kin Lok (黃健樂)             	 2709.16 Hong Kong      
  12 Mattia Furlan                  	 2702.26 Italy          
  13 Lin Chen (陈霖)                  	 2655.11 China          
  14 Carlos Tapp                    	 2622.70 Spain          
  15 Aryan Kejriwal                 	 2615.54 USA            
  16 Za

StringIndexError: StringIndexError("Asia Konvittayayotin (เอเชีย กรวิทยโยธิน)", 30)

In [91]:
printRatings(RatingList) # 5x5 Cube (35,0.018)


Rank Name                           	 Rating  Citizen of     
   1 Max Park                       	 3475.62 USA            
   2 Feliks Zemdegs                 	 3454.30 Australia      
   3 Sebastian Weyer                	 3336.41 Germany        
   4 Ciarán Beahan                  	 3232.82 Ireland        
   5 Kevin Hays                     	 3194.67 USA            
   6 Seung Hyuk Nahm (남승혁)          	 3143.96 Korea          
   7 Mats Valk                      	 3069.01 Netherlands    
   8 Patrick Ponce                  	 3036.03 USA            
   9 Bill Wang                      	 3020.56 Canada         
  10 Kevin Gerhardt                 	 2956.25 Germany        
  11 Dario Roa Sánchez              	 2951.06 Spain          
  12 Michał Halczuk                 	 2934.57 Poland         
  13 Philipp Weyer                  	 2906.98 Germany        
  14 Lucas Wesche                   	 2901.85 Germany        
  15 Pedro Alejandro Condo Tellez   	 2898.84 Peru           
  16 Ab

StringIndexError: StringIndexError("Asia Konvittayayotin (เอเชีย กรวิทยโยธิน)", 30)

In [51]:
printRatings(RatingList) # 3x3 Cube (35,0.006)


Rank Name                           	 Rating  Citizen of     
   1 Max Park                       	 3520.94 USA            
   2 Feliks Zemdegs                 	 3456.54 Australia      
   3 Philipp Weyer                  	 3357.24 Germany        
   4 Bill Wang                      	 3223.01 Canada         
   5 Seung Hyuk Nahm (남승혁)          	 3204.45 Korea          
   6 Lucas Etter                    	 3194.87 USA            
   7 Sebastian Weyer                	 3179.34 Germany        
   8 Patrick Ponce                  	 3154.13 USA            
   9 Mats Valk                      	 3116.90 Netherlands    
  10 Kevin Gerhardt                 	 3060.47 Germany        
  11 Zibo Xu (徐子博)                  	 3057.62 China          
  12 Alexandre Carlier              	 3048.59 France         
  13 Ciarán Beahan                  	 3010.86 Ireland        
  14 Michał Rzewuski                	 3003.45 Poland         
  15 Phillip Lewicki                	 3003.08 USA            
  16 Co

StringIndexError: StringIndexError("Nikita Kovalchuk (Нікіта Ковальчук)", 30)

In [97]:
sum = 0
for i in 1:length(RatingList)
    sum += RatingList[i][1]
end
println("mean: ", sum/length(RatingList))
println("median: ", RatingList[Int.(length(RatingList)/2)][1])
println("min: ", RatingList[length(RatingList)][1])
println("max: ", RatingList[1][1])

mean: 1500.8822854757689
median: 1500.0
min: 890.0193073463115
max: 3055.3553160980964
